In [1]:
from bs4 import SoupStrainer
import getpass
import os
from dataclasses import dataclass
import logging
from typing import List

import faiss
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader

# -- MultiQueryRetriever (LangChain) 
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts.prompt import PromptTemplate

# For the custom multi-query generation approach
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# For Reranking
from sentence_transformers import CrossEncoder
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_PROJECT"] = "RAG-MultiQuery-Reranker"

## Logging, Config 설정

In [4]:
# Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s - %(message)s"
)
logger = logging.getLogger("RAG-MultiQuery-Reranker")

In [43]:
@dataclass
class RAGConfig:
    """
    Holds configuration for the RAG pipeline.
    """
    data_dir: str = "./naver_stock_data"  # Directory with raw texts
    chunk_size: int = 1000
    chunk_overlap: int = 100
    embedding_model: str = "intfloat/multilingual-e5-large-instruct"
    vector_store_dir: str = "./chroma_naver_stock"
    openai_model_name: str = "gpt-4o-mini"  # or "gpt-4"
    multiquery_prompt_template: str = (
        "You are an AI assisting a financial analyst. The user query is:\n"
        "{question}\n"
        "Rewrite this single query into multiple distinct queries that focus on different aspects "
        "of the stock/financial domain. At least 3 variants, ensuring coverage of:\n"
        "- Different time frames\n"
        "- Different possible financial terms or synonyms\n"
        "- Additional detail or context about the company or market trends\n"
        "You must create query by korean language and don't create blank query"
    )
    top_k: int = 30
    rerank_top_k: int = 5
    rerank_model_name: str = "cross-encoder/ms-marco-MiniLM-L-12-v2"

## Document Index 생성

In [7]:
# embedding 모델 로드
embedding_fn = HuggingFaceEmbeddings(model_name=RAGConfig.embedding_model)
dimension_size = len(embedding_fn.embed_query("hello world"))
print(dimension_size)

2025-01-30 11:58:06,464 [INFO] sentence_transformers.SentenceTransformer - Use pytorch device_name: cuda
2025-01-30 11:58:06,464 [INFO] sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large-instruct


1024


In [17]:
# Create new FAISS vectorstore
vector_db = FAISS(
    embedding_function=embedding_fn,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# If you already have local faiss, just load it
# vector_db = FAISS.load_local("./vectorstore/naver_news_faiss", index_name="index", embeddings=embedding_fn,
#                              allow_dangerous_deserialization=True)

In [89]:
# crawling finance news from naver
from datetime import datetime, timedelta
from naver_news_crawling import extract_num_pagination, extract_news_links, HEADERS

def add_news_into_db():
    daily_news_link_prefix = "https://finance.naver.com/news/mainnews.naver?date="
    # 오늘 날짜
    start_date = datetime(2025, 1, 1)
    end_date = datetime.today()
    cur_date = start_date
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    while cur_date <= end_date:
        date_string = cur_date.strftime("%Y-%m-%d")
        daily_news_link = daily_news_link_prefix + date_string

        last_page = extract_num_pagination(daily_news_link)
        for i in range(int(last_page)):
            page_news_link = daily_news_link + f"&page={i + 1}"
            news_link_list = extract_news_links(page_news_link)
            loader = WebBaseLoader(
                web_paths=news_link_list,
                bs_kwargs=dict(
                    parse_only=SoupStrainer(
                        "div",
                        attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
                    )
                ),
                header_template=HEADERS
            )
            docs = loader.load()
            for doc in docs:
                doc.metadata['date'] = date_string
            split_docs = text_splitter.split_documents(docs)
            vector_db.add_documents(split_docs)
            print(f"URL: {page_news_link} / num of docs: {len(docs)} / num of chunks: {len(split_docs)}")
        cur_date += timedelta(days=1)

In [90]:
add_news_into_db()

URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-01&page=1 / num of docs: 20 / num of chunks: 45
URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-01&page=2 / num of docs: 20 / num of chunks: 49
URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-01&page=3 / num of docs: 20 / num of chunks: 52
URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-01&page=4 / num of docs: 20 / num of chunks: 56
URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-01&page=5 / num of docs: 20 / num of chunks: 80
URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-01&page=6 / num of docs: 4 / num of chunks: 17
URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-02&page=1 / num of docs: 20 / num of chunks: 47
URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-02&page=2 / num of docs: 20 / num of chunks: 62
URL: https://finance.naver.com/news/mainnews.naver?date=2025-01-02&page=3 / num of docs: 20 / num of chun

In [19]:
vector_db.save_local("./vectorstore/naver_news_faiss", index_name="2025_january")

## Multi Query Retriever

In [44]:
def get_multiquery_retriever(vectorstore) -> MultiQueryRetriever:
    """
    Uses LangChain's MultiQueryRetriever to create multiple queries from a single user query.
    """
    # We define a prompt that clarifies how to generate sub-queries
    mq_prompt = PromptTemplate(
        input_variables=["query"],
        template=RAGConfig.multiquery_prompt_template
    )
    # LLM for generating multi-queries
    llm = ChatOpenAI(
        model_name=RAGConfig.openai_model_name,
        temperature=1.0
    )

    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=vectorstore.as_retriever(search_kwargs={"k": RAGConfig.top_k}),
        llm=llm,
        prompt=mq_prompt
    )
    return multiquery_retriever

mq_retriever = get_multiquery_retriever(vector_db)

In [41]:
mq_retriever.invoke("삼성전자의 최근 주가가 하락세인 이유에 대해 알려줘")

2025-01-30 12:35:31,324 [INFO] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-30 12:35:31,336 [INFO] langchain.retrievers.multi_query - Generated queries: ['1. 삼성전자의 최근 주가 하락 원인에 대한 분석과 시장 반응은 어떤가요?', '2. 삼성전자의 주가가 지난 6개월 동안 하락한 이유와 그에 따른 재무적 영향은 무엇인가요?', '3. 삼성전자 주가 하락과 관련된 글로벌 경제 동향이나 산업 내 경쟁 상황은 어떻게 변화하고 있나요?']


[Document(id='bcf1cb20-b269-4178-a4ab-5286eb4f201c', metadata={'source': 'https://n.news.naver.com/mnews/article/001/0015149187', 'date': '2025-01-08'}, page_content='풀이된다.    이경민 대신증권 연구원은 "낮아질 대로 낮아진 기대치로 인해 실적이 예상보다 부진했음에도 불구하고 강한 안도 심리를 자극한 것으로 볼 수 있다"며 "삼성전자 주가 수준이 고점 대비 40% 이상 하락하는 등 역사적 저점권에 위치함에 따라 이런 역발상적 주가 흐름이 가능했다고 판단된다"고 말했다.    josh@yna.co.kr'),
 Document(id='0cb7f736-2938-4e24-94b3-c0915056a00e', metadata={'source': 'https://n.news.naver.com/mnews/article/119/0002910180', 'date': '2025-01-03'}, page_content='삼성전자 서초사옥 전경. ⓒ데일리안 DB[데일리안 = 황인욱 기자] 지난해 글로벌 증시 대비 국내 증시의 부진이 두드러졌던 한 요인으로 삼성전자의 하락세가 지목된다. 새해 코스피지수가 상승 국면에 들기 위한 조건으로 삼성전자의 반등세가 절실하나 반도체 업황 우려가 여전해 투심 회복에 시간이 걸릴 것이라는 관측이 나온다.3일 한국거래소에 따르면 작년 말 기준 코스피 시가총액은 1963조3288억원으로 연초(2024년 1월 2일 기준) 2147조2239억원 대비 8.56%(183조8951억원) 감소했다. 이 기간 삼성전자 시총은 475조1957억원에서 319조3834억원으로 32.79%(155조8123억원) 줄었다.작년 코스피 전체 시총 감소분에서 삼성전자가 차지하는 비중은 84.73%에 달했다. 시총 1위 삼성전자의 주가가 32.23%(7만8500→5만3200원)로 하락하며 지수도 9.63%(2655.28→2399.49) 내렸다.삼성

In [46]:
# ----------------------------------------------------------
# 4. Reranker (CrossEncoder)
# ----------------------------------------------------------
class CustomCrossEncoderReranker:
    """
    Simple Cross-Encoder-based reranker that re-scores 
    (query, doc) pairs and sorts by highest relevance.
    """

    def __init__(self, model_name: str):
        logger.info(f"Initializing CrossEncoder reranker with: {model_name}")
        self.rerank_model = CrossEncoder(model_name)

    def rerank(self, query: str, docs: List[Document], top_k: int = RAGConfig.rerank_top_k) -> List[Document]:
        """
        Re-ranks the docs by CrossEncoder score. Returns top_n documents.
        """
        # Prepare pairs
        pair_inputs = [(query, d.page_content) for d in docs]
        scores = self.rerank_model.predict(pair_inputs)

        # Sort by descending score
        idx_scores = list(enumerate(scores))
        idx_scores.sort(key=lambda x: x[1], reverse=True)
        top_docs = [docs[i] for (i, s) in idx_scores[:top_k]]

        logger.info("Reranker top doc scores:")
        for i, (idx, s) in enumerate(idx_scores[:top_k]):
            logger.info(f"{i + 1}) Score={s:.4f}, doc source={docs[idx].metadata.get('source', '')}")
        return top_docs
    
reranker = CustomCrossEncoderReranker("cross-encoder/ms-marco-MiniLM-L-12-v2")
user_query = "미국의 AI 데이터센터 뉴스와 관련되어 주목할만한 테마가 있어?"
mq_docs = mq_retriever.invoke(user_query)
rerank_docs = reranker.rerank(user_query, mq_docs)

2025-01-30 12:43:05,146 [INFO] RAG-MultiQuery-Reranker - Initializing CrossEncoder reranker with: cross-encoder/ms-marco-MiniLM-L-12-v2
2025-01-30 12:43:05,624 [INFO] sentence_transformers.cross_encoder.CrossEncoder - Use pytorch device: cuda
2025-01-30 12:43:07,699 [INFO] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-30 12:43:07,702 [INFO] langchain.retrievers.multi_query - Generated queries: ['1. 최근 3개월간 미국 AI 데이터센터와 관련된 주요 기업의 주가 변동은 어땠나요?  ', '2. 2024년도 미국 AI 데이터센터 투자 전망에 대한 분석과 관련된 주요 경제 지표는 무엇인가요?  ', '3. 미국의 AI 데이터센터 시장 성장률과 이로 인한 관련 기업들의 수익성 변화에 대한 장기적인 트렌드는 어떤가요?']


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2025-01-30 12:43:08,164 [INFO] RAG-MultiQuery-Reranker - Reranker top doc scores:
2025-01-30 12:43:08,164 [INFO] RAG-MultiQuery-Reranker - 1) Score=8.2034, doc source=https://n.news.naver.com/mnews/article/009/0005435367
2025-01-30 12:43:08,165 [INFO] RAG-MultiQuery-Reranker - 2) Score=8.1473, doc source=https://n.news.naver.com/mnews/article/016/0002411373
2025-01-30 12:43:08,165 [INFO] RAG-MultiQuery-Reranker - 3) Score=7.9541, doc source=https://n.news.naver.com/mnews/article/009/0005431897
2025-01-30 12:43:08,165 [INFO] RAG-MultiQuery-Reranker - 4) Score=7.9413, doc source=https://n.news.naver.com/mnews/article/011/0004444337
2025-01-30 12:43:08,165 [INFO] RAG-MultiQuery-Reranker - 5) Score=7.9374, doc source=https://n.news.naver.com/mnews/article/018/0005930350


In [47]:
from langchain.retrievers.document_compressors import CrossEncoderReranker

rerank_model = HuggingFaceCrossEncoder(model_name="Dongjin-kr/ko-reranker")
compressor = CrossEncoderReranker(model=rerank_model, top_n=3)

comp_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=mq_retriever
)
comp_retriever.invoke("삼성전자가 최근에 주가가 떨어지는 이유는 뭐야?")

2025-01-30 12:43:45,071 [INFO] sentence_transformers.cross_encoder.CrossEncoder - Use pytorch device: cuda


In [48]:
comp_retriever.invoke("삼성전자가 최근에 주가가 떨어지는 이유는 뭐야?")

2025-01-30 12:44:01,374 [INFO] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-30 12:44:01,380 [INFO] langchain.retrievers.multi_query - Generated queries: ['1. 삼성전자의 최근 분기 실적 발표가 주가 하락에 미친 영향은 무엇인가요?', '2. 최근 1년 동안 삼성전자의 주가 변동과 관련된 외부 요인은 어떤 것들이 있나요?', '3. 삼성전자의 경쟁 업체와 비교했을 때, 최근 주가 하락의 원인은 무엇인지 분석해 주세요.']


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

[Document(id='0cb7f736-2938-4e24-94b3-c0915056a00e', metadata={'source': 'https://n.news.naver.com/mnews/article/119/0002910180', 'date': '2025-01-03'}, page_content='삼성전자 서초사옥 전경. ⓒ데일리안 DB[데일리안 = 황인욱 기자] 지난해 글로벌 증시 대비 국내 증시의 부진이 두드러졌던 한 요인으로 삼성전자의 하락세가 지목된다. 새해 코스피지수가 상승 국면에 들기 위한 조건으로 삼성전자의 반등세가 절실하나 반도체 업황 우려가 여전해 투심 회복에 시간이 걸릴 것이라는 관측이 나온다.3일 한국거래소에 따르면 작년 말 기준 코스피 시가총액은 1963조3288억원으로 연초(2024년 1월 2일 기준) 2147조2239억원 대비 8.56%(183조8951억원) 감소했다. 이 기간 삼성전자 시총은 475조1957억원에서 319조3834억원으로 32.79%(155조8123억원) 줄었다.작년 코스피 전체 시총 감소분에서 삼성전자가 차지하는 비중은 84.73%에 달했다. 시총 1위 삼성전자의 주가가 32.23%(7만8500→5만3200원)로 하락하며 지수도 9.63%(2655.28→2399.49) 내렸다.삼성전자의 주가 하락은 외국인 수급 이탈 영향이 컸다. 외국인은 작년 한 해(2024.1.2~2024.12.30) 동안 코스피 주식을 2조7464억원 순매수하면서도 삼성전자는 10조5197억원이나 순매도했다.특히 외국인은 작년 9월부터 연말까지 순매도 기조를 보이며 지수 하방 압력을 높였는데 이 기간 외국인의 삼성전자 순매도 규모는 19조1979억원으로 코스피 순매도 19조1416억원을 상회했다. 사실상 삼성전자에만 매도 주문을 쏟아낸 셈이다.박소연 신영증권 연구원은 “작년 외국인 순매매 동향을 집계해보면 특이한 현상이 하나 관찰되는데 삼성전자 한 종목을 제외하면 외국인은 한국시장에서 12조원 가까이를 순매수했다는 점”이라며 “12월 탄핵 정국

In [50]:
fin_analysis_template = """
[시스템 역할 / 설정]
당신은 “투자 전문가”이다. 주식·투자 전반에 대한 폭넓은 지식과 인사이트를 갖추고 있으며, 
증권 뉴스나 관련 문헌(Context)을 빠르게 분석할 수 있다.

[지시사항]
1. 질문 유형 구분
   - "분석" 질문: 미래 투자 포인트, 주가 동향, 향후 전망과 그 이유를 설명
   - "검색" 질문: 주어진 context 범위 안에서 정확한 사실(fact)을 기술
   
2. 답변 방식
   - 만약 질문이 "분석" 유형이라면:
     - (a) LLM이 이미 알고 있는 금융 지식(in-memory)과
     - (b) 위 Context(증권 뉴스)를 종합해,
     - **미래전망**이나 **투자 기회**, **재무적 분석**, **시장 트렌드** 등을 제시한다.
     - 중요한 근거(뉴스 속 정보, 과거 데이터 등)를 요약적으로 포함하고,
     - 예측에 대한 논리와 이유를 구체적으로 설명한다.
   - 만약 질문이 "검색" 유형이라면:
     - (a) 위 Context 내 명시된 정보에 집중하며,
     - (b) 확인된 사실만 전달한다 (추측·예측은 삼가기).
     - 특정 주가 변동, 공시 정보, 뉴스에 언급된 사실 등에 대해서만 언급하고,
     - 맥락 밖 추론은 피한다.

3. 답변 작성 시 유의사항
   - “투자 전문가”의 관점에서 **신뢰성** 있고 **명확한 근거**를 제시한다.
   - 필요 시 문장 말미에 “예측일 뿐이며, 실제 투자 판단은 본인의 책임”이라는 식의 간단한 주의 문구를 포함해도 좋다.
   - 답변을 최대한 간결하고 핵심만 요약하되, 질문이 분석을 요구한다면 **전문적인 어휘**와 **근거**를 충분히 제시한다.

[최종 출력 (Answer)]
위 지시사항과, "투자 전문가"라는 역할, 그리고 두 가지 질문 유형에 따른 답변 방식을 적용하여 사용자 질문에 대한 답변을 완성하세요.

[사용자 질문 (User Query)]
{question}

[주어진 Context (증권 뉴스)]
{context}
"""

prompt = PromptTemplate.from_template(fin_analysis_template)

In [51]:
generator = ChatOpenAI(model_name=RAGConfig.openai_model_name, temperature=0)

rag_chain = (

        {"context": comp_retriever, "question": RunnablePassthrough()}
        | prompt
        | generator
        | StrOutputParser()
        | (lambda x: x.split('\n'))
)

In [53]:
output = rag_chain.invoke("삼성전자 지금 사도 되나요?")

2025-01-30 12:52:26,654 [INFO] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-30 12:52:26,668 [INFO] langchain.retrievers.multi_query - Generated queries: ['1. 삼성전자의 현재 주가가 앞으로 1개월 동안 상승할 가능성이 얼마나 될까요?', '2. 삼성전자의 PER(주가수익비율)은 현재 얼마이며, 경쟁사 대비 어떠한가요?', '3. 최근 삼성전자에 대한 시장 전망이나 애널리스트의 추천이 어떻게 변화했나요?']


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2025-01-30 12:52:35,365 [INFO] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [55]:
print('\n'.join(output))

삼성전자의 현재 주가에 대한 투자 결정을 고려할 때, 여러 요소를 분석해볼 필요가 있습니다.

1. **주가 현황**: 삼성전자는 최근 5만4300원에 거래되고 있으며, 이는 지난해 1월 22일 종가인 7만5100원과 비교해 28% 하락한 수치입니다. 또한, 반년 전 52주 최고가인 8만8800원 대비 39% 낮은 가격입니다.

2. **외국인 투자자 매도**: 외국인 투자자들이 올해 들어 삼성전자를 대규모로 매도하고 있으며, 이는 주가 하락의 주요 원인 중 하나입니다. 1월 2일부터 21일 사이에만 7603억원어치를 순매도했습니다.

3. **실적 및 전망**: 삼성전자는 지난해 4분기 연결 매출액 75조원, 영업이익 6조5000억원을 기록했으나, 전 분기 대비 각각 5.18%, 29.19% 감소했습니다. 이는 시장 기대치를 밑돌며 부진한 실적을 나타냅니다. 그러나 전문가들은 현재 주가가 '딥밸류' 구간에 있으며, 저가 매수 전략이 유효하다고 평가하고 있습니다.

4. **미래 전망**: 증권가에서는 올해 하반기부터 실적 개선이 예상되며, '갤럭시 S25' 발매와 메모리 반도체 수급 개선이 주요 상승 동력으로 작용할 것으로 보입니다. 또한, 현재 주가는 대부분의 악재를 반영한 상태로, 작은 호재에도 민감하게 반응할 가능성이 있습니다.

결론적으로, 삼성전자는 현재 저가 매수 기회로 평가받고 있으며, 향후 실적 개선이 기대되는 상황입니다. 그러나 외국인 투자자의 매도세와 부진한 실적이 여전히 주가에 부담을 주고 있으므로, 신중한 접근이 필요합니다. 

**예측일 뿐이며, 실제 투자 판단은 본인의 책임입니다.**


In [ ]:
# ----------------------------------------------------------
# 3-1. Multi-Query generation with a local HF model + prompt
# ----------------------------------------------------------
def generate_multi_queries_locally(user_query: str) -> List[str]:
    """
    Example of using a local Hugging Face model (causal LM or instruct model)
    to produce multi-queries. 
    """
    # For demonstration, use a small model. In reality, 
    # you might want a more capable instruct or T5 model, e.g. 'google/flan-t5-base'
    local_model_name = "microsoft/GPT-2-1.5B-125M"  # (hypothetical small model)
    # If the model doesn't exist, replace with a real HF model
    tokenizer = AutoTokenizer.from_pretrained(local_model_name)
    model = AutoModelForCausalLM.from_pretrained(local_model_name)
    text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

    # A domain-specific prompt
    local_prompt = (
        f"사용자 질의(증권/주식 관련): '{user_query}'\n\n"
        "위 질의를 재구성하여, 서로 다른 관점과 핵심 용어를 추가한 최소 3개의 유사 질의를 만들어 주세요.\n"
        "각 질의는 증권/금융 용어를 적극 활용하고, 서로 겹치지 않도록 최대한 다르게 표현합니다:\n\n"
    )
    response = text_gen(local_prompt, max_length=256, num_return_sequences=1, do_sample=True)
    # parse out the queries
    # For simplicity, we assume the model returns them in a list form or separated lines
    text_out = response[0]["generated_text"]
    # naive split by newlines
    candidate_lines = [line.strip() for line in text_out.split("\n") if line.strip() != ""]

    # Filter to get only the lines that look like queries
    multi_queries = []
    for line in candidate_lines:
        # A simple heuristic
        if "?" in line or "질의:" in line:
            multi_queries.append(line)
    # If we didn't parse well, fallback
    if len(multi_queries) < 3:
        multi_queries = candidate_lines[-3:]  # last 3 lines

    logger.info("Local HF-based multi-queries:")
    for mq in multi_queries:
        logger.info(f"- {mq}")
    return multi_queries